# Cosine similarity between TikTok transcripts

**Author**: Catherine Foster \
**Date**: 04/22/24

In [41]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [97]:
#read in CSV with video links
link_df = pd.read_csv('filtered_merged_recommended.csv')#.drop(columns=['current_time'])

#Read in the CSV with transcriptions
transcript_df = pd.read_csv('english_transcriptions.csv', names=['vid_id', 'transcription'])

#prep output file
cosinesim_df = pd.DataFrame(columns=['distributed vid_id', 'persona', 'current_time', 'rec1_cosim',
                                    'rec2_cosim', 'rec3_cosim', 'rec4_cosim', 
                                     'rec5_cosim', 'rec6_cosim', 'rec7_cosim', 'rec8_cosim'])
link_df

,distributed_link,persona,current_time,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8
0,https://www.tiktok.com/@evilnetik/video/732839...,control,2024-04-17 21:30:59.842660,https://www.tiktok.com/@paytonhughes18/video/7...,https://www.tiktok.com/@kristy.jo.hinojosa/vid...,https://www.tiktok.com/@sarasplansky/video/734...,https://www.tiktok.com/@michael_liscio72/video...,https://www.tiktok.com/@jeremy_littel/video/73...,https://www.tiktok.com/@andiebush/video/734603...,https://www.tiktok.com/@urting.pissy.flame/vid...,https://www.tiktok.com/@wefxbmk/video/73347359...
1,https://www.tiktok.com/@allman289/video/735131...,control,2024-04-17 22:00:00,https://www.tiktok.com/@say_luhm/video/7331457...,https://www.tiktok.com/@watsonschocolates/vide...,https://www.tiktok.com/@maryyycarol/video/7350...,https://www.tiktok.com/@theaquabreeze/video/73...,https://www.tiktok.com/@limbatrip/video/734274...,https://www.tiktok.com/@jamal.0340/video/73506...,https://www.tiktok.com/@lucynaidun/video/73519...,https://www.tiktok.com/@2raree/video/734326887...
2,https://www.tiktok.com/@thatfoodielyss/video/7...,control,2024-04-18 02:00:00,https://www.tiktok.com/@ployppatra/video/73581...,https://www.tiktok.com/@j.billiard3/video/7344...,https://www.tiktok.com/@j.billiard3/video/7352...,https://www.tiktok.com/@milo.504.23/video/7353...,https://www.tiktok.com/@boxingalleymike/video/...,https://www.tiktok.com/@islandgal_5homedecor/v...,https://www.tiktok.com/@j.billiard3/video/7336...,https://www.tiktok.com/@thatfoodielyss/video/7...
3,https://www.tiktok.com/@meme._.galaxy/video/73...,control,2024-04-18 03:00:00,https://www.tiktok.com/@ring_jedi/video/734778...,https://www.tiktok.com/@haruun_apdi/video/7348...,https://www.tiktok.com/@manualidades_diy_fei/v...,https://www.tiktok.com/@ring_jedi/video/734503...,https://www.tiktok.com/@off.axl/video/73281967...,https://www.tiktok.com/@devid4936/video/732759...,https://www.tiktok.com/@youstylewb/video/73425...,https://www.tiktok.com/@wiwigato/video/7329216...
4,https://www.tiktok.com/@chhloekim/video/735386...,control,2024-04-18 04:00:00,https://www.tiktok.com/@chhloekim/video/733145...,https://www.tiktok.com/@chhloekim/video/735416...,https://www.tiktok.com/@inumakimmi/video/73536...,https://www.tiktok.com/@chhloekim/video/734108...,https://www.tiktok.com/@cindywon/video/7345345...,https://www.tiktok.com/@camfant/video/73530310...,https://www.tiktok.com/@cherry_keem/video/7357...,https://www.tiktok.com/@monakoaa/video/7350306...
...,...,...,...,...,...,...,...,...,...,...,...
187,https://www.tiktok.com/@gordonramsayofficial/v...,Beauty,2024-04-21 18:00:15.516060,https://www.tiktok.com/@sixflags/video/7345930...,https://www.tiktok.com/@benbuttv/video/7335213...,https://www.tiktok.com/@kelly.toto/video/73514...,https://www.tiktok.com/@official_nct/video/733...,https://www.tiktok.com/@miniiiartist/video/736...,https://www.tiktok.com/@jern/video/73601975254...,https://www.tiktok.com/@babymonster.base/video...,https://www.tiktok.com/@kelly.toto/video/73500...
188,https://www.tiktok.com/@chabewhite/video/73551...,Beauty,2024-04-21 19:00:03.289645,https://www.tiktok.com/@chabewhite/video/73343...,https://www.tiktok.com/@chabewhite/video/73597...,https://www.tiktok.com/@chabewhite/video/73375...,https://www.tiktok.com/@chabewhite/video/73384...,https://www.tiktok.com/@guangsdxy4z/video/7359...,https://www.tiktok.com/@chabewhite/video/73408...,https://www.tiktok.com/@chabewhite/video/73341...,https://www.tiktok.com/@octpath/video/73424793...
189,https://www.tiktok.com/@minibakerychannel/vide...,Beauty,2024-04-21 22:00:00.526748,https://www.tiktok.com/@crazycae/video/7329985...,https://www.tiktok.com/@pmdamiann/video/735616...,https://www.tiktok.com/@pmdamiann/video/732970...,https://www.tiktok.com/@kashtonparker/video/73...,https://www.tiktok.com/@gabyluvsyousm/video/73...,https://www.tiktok.com/@onlyfamousbrian/video/...,https://www.tiktok.com/@olybre/video/734546882...,https://www.tiktok.com/@pmdamiann/video/7332

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [98]:
count = 0

In [99]:
seen_links = []
for index, (link, date) in link_df.loc[:, ['distributed_link', 'current_time']].iterrows():
    distributed_link = link
    distributed_id = (re.findall(r'video/(\d+)', distributed_link))[0]
    #print(distributed_id)
    if distributed_link not in seen_links:
        seen_links.append(distributed_link)
        if distributed_id in transcript_df['vid_id'].values:
            distributed_transcript = transcript_df.loc[transcript_df['vid_id'] == distributed_id, 'transcription'].iloc[0]
            if isinstance(distributed_transcript, str):
                count+=1
                distributed_embeddings = model.encode(distributed_transcript, convert_to_tensor=True)
                for category in ['control', 'Society', 'Sports', 'Beauty']:
                    cosim_list = [distributed_id, category, date]

                    query_string = f"distributed_link == @distributed_link and persona == @category"
                    rec_links = link_df.query(query_string, inplace=False)
                    rec1_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_1']))[0] 
                    rec2_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_2']))[0] 
                    rec3_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_3']))[0] 
                    rec4_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_4']))[0] 
                    rec5_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_5']))[0] 
                    rec6_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_6']))[0] 
                    rec7_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_7']))[0] 
                    rec8_id = (re.findall(r'video/(\d+)', rec_links.iloc[0]['rec_8']))[0] 
                    rec_ids = [rec1_id, rec2_id, rec3_id, rec4_id, rec5_id, rec6_id, rec7_id, rec8_id]
                    for rec_id in rec_ids:
                        if rec_id in transcript_df['vid_id'].values:
                            rec_transcript = transcript_df.loc[transcript_df['vid_id'] == rec_id, 'transcription'].iloc[0]
                            if isinstance(rec_transcript, str):
                                rec_embeddings = model.encode(rec_transcript, convert_to_tensor=True)
                                cosine_score = (util.cos_sim(distributed_embeddings, rec_embeddings)).item()
                                cosim_list.append(cosine_score)
                            else: 
                                cosim_list.append(None)
                        else:
                            cosim_list.append(None)
                    new_row_df = pd.DataFrame([cosim_list], columns=cosinesim_df.columns)
                    cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cosinesim_df = pd.concat([cosinesim_df, new_row_df], axis=0)
/var/folders/n_/lld31hms2tzffwdqgcdg20jc0000gn/T/ipykernel_5057/1025502006.py:39: FutureWarning: The behavior of DataFrame

In [100]:
count

29

In [101]:
cosinesim_df

,distributed vid_id,persona,current_time,rec1_cosim,rec2_cosim,rec3_cosim,rec4_cosim,rec5_cosim,rec6_cosim,rec7_cosim,rec8_cosim
0,7328399559120424235,control,2024-04-17 21:30:59.842660,NaN,0.018224,0.005321,-0.047734,0.269092,NaN,0.119745,0.280983
0,7328399559120424235,Society,2024-04-17 21:30:59.842660,0.147348,NaN,0.066227,0.124199,0.103538,0.192235,NaN,0.169935
0,7328399559120424235,Sports,2024-04-17 21:30:59.842660,-0.047734,0.033899,0.077328,0.134838,0.016674,0.047015,NaN,0.132819
0,7328399559120424235,Beauty,2024-04-17 21:30:59.842660,0.261162,0.142988,0.044972,0.123249,0.077065,0.193717,0.025986,-0.010765
0,7334316740840574240,control,2024-04-18 03:00:00,0.226828,NaN,0.193422,0.226828,0.039468,NaN,0.385137,NaN
...,...,...,...,...,...,...,...,...,...,...,...
0,7327423854966557998,Beauty,2024-04-21 15:00:00,NaN,NaN,0.217045,0.075302,0.240656,NaN,NaN,NaN
0,7341108653442829601,control,2024-04-21 18:00:00,NaN,0.010591,0.290612,NaN,0.198181,0.156102,NaN,0.241713
0,7341108653442829601,Society,2024-04-21 18:00:00,0.111548,NaN,-0.005710,NaN,NaN,0.050488,0.253591,0.051713
0,7341108653442829601,Sports,2024-04-21 18:00:00,0.290612,0.010591,0.156102,0.241713,0.077966,NaN,NaN,0.313420


In [102]:
#cosinesim_df['avg_cosim_allrecs'] = cosinesim_df.mean(axis=1)
cosinesim_df['avg_cosim_allrecs'] = cosinesim_df.loc[:, ["rec1_cosim","rec2_cosim","rec3_cosim", "rec4_cosim",
                                   "rec5_cosim", "rec6_cosim", "rec7_cosim", "rec8_cosim"]].mean(axis = 1)

In [103]:
cosinesim_df

,distributed vid_id,persona,current_time,rec1_cosim,rec2_cosim,rec3_cosim,rec4_cosim,rec5_cosim,rec6_cosim,rec7_cosim,rec8_cosim,avg_cosim_allrecs
0,7328399559120424235,control,2024-04-17 21:30:59.842660,NaN,0.018224,0.005321,-0.047734,0.269092,NaN,0.119745,0.280983,0.107605
0,7328399559120424235,Society,2024-04-17 21:30:59.842660,0.147348,NaN,0.066227,0.124199,0.103538,0.192235,NaN,0.169935,0.133914
0,7328399559120424235,Sports,2024-04-17 21:30:59.842660,-0.047734,0.033899,0.077328,0.134838,0.016674,0.047015,NaN,0.132819,0.056406
0,7328399559120424235,Beauty,2024-04-17 21:30:59.842660,0.261162,0.142988,0.044972,0.123249,0.077065,0.193717,0.025986,-0.010765,0.107297
0,7334316740840574240,control,2024-04-18 03:00:00,0.226828,NaN,0.193422,0.226828,0.039468,NaN,0.385137,NaN,0.214337
...,...,...,...,...,...,...,...,...,...,...,...,...
0,7327423854966557998,Beauty,2024-04-21 15:00:00,NaN,NaN,0.217045,0.075302,0.240656,NaN,NaN,NaN,0.177668
0,7341108653442829601,control,2024-04-21 18:00:00,NaN,0.010591,0.290612,NaN,0.198181,0.156102,NaN,0.241713,0.179440
0,7341108653442829601,Society,2024-04-21 18:00:00,0.111548,NaN,-0.005710,NaN,NaN,0.050488,0.253591,0.051713,0.092326
0,7341108653442829601,Sports,2024-04-21 18:00:00,0.290612,0.010591,0.156102,0.241713,0.077966,NaN,NaN,0.313420,0.181734


In [104]:
cosinesim_df.to_csv('cosinesim_recs_vs_original.csv', index=False)